## Credit Risk Resampling Techniques

In [1]:
# Import dependencies 

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Import sklearn - an effecient and simple tool for predictive data analysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

# Import imblearn - imbalanced learning to deal with classification with imbalanced classes
from imblearn.metrics import classification_report_imbalanced

from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.under_sampling import ClusterCentroids

## Read the CV into a DataFrame

In [2]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


## Split the data into training and testing

In [3]:
# Create our features

X = pd.get_dummies(df.drop('loan_status', axis =1))

# Create our target

y = df[['loan_status']]

In [18]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [19]:
y.describe()

,loan_status
count,77536
unique,2
top,low_risk
freq,75036


In [6]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [7]:
# Create the X-train,X_test, y_train, and y_test 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train.shape

(58152, 10)

## Data Pre-processing

>Scale the training and testing data using the StandardScaler from sklearn. Remember that when scaling the data, 
you only scale the features data (X_train and X_testing).

In [8]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler

object = StandardScaler()

In [9]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset

scale = object.fit_transform(X_train)

In [10]:
# Scale the training and testing data

scaled_train = object.transform(X_train)

scaled_test = object.transform(X_test)

## Simple logistic Regression



In [11]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)


LogisticRegression(random_state=1)

In [12]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9520479254722232

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[  563,    56],
       [  102, 18663]])

In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.91      0.99      0.88      0.95      0.90       619
   low_risk       1.00      0.99      0.91      1.00      0.95      0.91     18765

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



## Oversampling

>In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. 
You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. 
For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using Counter from the collections library.
2. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the imbalanced_classification_report from imbalanced-learn.

-Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [15]:
# Resample the training data with the RandomOversampler
model = RandomOverSampler(random_state=1)
X_resampled, y_resampled = model.fit_resample(X_train, y_train)
print(model)

# View the count of target classes with Counter
Counter(y_resampled)

RandomOverSampler(random_state=1)


Counter({'loan_status': 1})

In [16]:
# Train the Logistic Regression model using the resampled data

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)


LogisticRegression(random_state=1)

In [22]:
# Calculated the balanced accuracy score
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions
                        }).reset_index(drop=True)

balanced_accuracy_score(y_test,predictions)

0.9936781215845847

In [24]:
# Display the confusion matrix
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions}).reset_index(drop=True)
confusion_matrix(y_test, predictions)

array([[  615,     4],
       [  116, 18649]])

In [25]:
# Print the imbalanced classification report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

   high_risk       0.84      0.99      0.91       619
    low_risk       1.00      0.99      1.00     18765

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### SMOTE Oversampling

In [26]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1).fit_resample(
    X_train, y_train
)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [27]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [28]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9936781215845847

In [29]:
# Display the confusion matrix
pd.DataFrame(
    confusion_matrix(y_test, y_pred), 
    index=["Hgih Risk", "Low Risk"], 
    columns=["Positive", "Negative"]
)

,Positive,Negative
Hgih Risk,615,4
Low Risk,116,18649


In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



## Undersampling

>In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the 
oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using Counter from the collections library.

2. Use the resampled data to train a logistic regression model.

3. Calculate the balanced accuracy score from sklearn.metrics.

4. Print the confusion matrix from sklearn.metrics

5. Generate a classication report using the imbalanced_classification_report from imbalanced-learn.

-Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [32]:
# Resample the data using the ClusterCentroids resampler

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)


Counter({'loan_status': 1})

In [33]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [34]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9881304218875842

In [35]:
# Display the confusion matrix
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=["Hgih Risk", "Low Risk"], 
    columns=["Positive", "Negative"]
)

,Positive,Negative
Hgih Risk,608,11
Low Risk,112,18653


In [36]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.98      0.99      0.91      0.99      0.98       619
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18765

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



## Combination (Over and Under) Sampling

>In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the
 best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm
  and complete the folliowing steps:

1. View the count of the target classes using Counter from the collections library.
2. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the imbalanced_classification_report from imbalanced-learn.


- Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests



In [37]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
Counter(y_resampled)


Counter({'loan_status': 1})

In [38]:
# Train the Logistic Regression model using the resampled data

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [39]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9935182494822666

In [40]:
# Display the confusion matrix
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=["Hgih Risk", "Low Risk"],
    columns=["Positive", "Negative"]
)

,Positive,Negative
Hgih Risk,615,4
Low Risk,122,18643


In [41]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



## Final Questions

1. Which model had the best balanced accuracy score?

> SMOTE and Naive Random oversampling models produced the best balanced accuracy score of 0.993

2. Which model had the best recall score? 

> All models produced same recall score of 0.99. I do think that there are issues with the data modeling here !

3. Which model had the best geometric mean score? 

> All models have the same geometric mean score of 0.99. Again, I think there are some issues in the data modeling !